# Local NIM Deployment with Semantic Kernel

This notebook demonstrates how to integrate a local NVIDIA Inference Microservice (NIM) deployment with Microsoft Semantic Kernel for AI-powered function calling and inference.

## What is NIM?

NVIDIA Inference Microservice (NIM) is a container-based inference solution that allows you to deploy and serve AI models locally or in the cloud. It provides:

- **High Performance**: Optimized for NVIDIA GPUs with TensorRT acceleration
- **Easy Deployment**: Containerized microservices for consistent deployment
- **OpenAI-Compatible API**: Standard REST API interface for seamless integration
- **Production Ready**: Built-in security, monitoring, and scaling capabilities

## Prerequisites

Before running this notebook, ensure you have:

1. **Local NIM deployment** running (e.g., LLaMA 3.1 8B Instruct model)
2. **Environment variables** configured:
   - `NIM_ENDPOINT_URL`: Your local NIM endpoint (e.g., `http://localhost:8000`)
   - `NIM_API_KEY`: API key for authentication (if required)
   - `AZURE_OPENAI_ENDPOINT`, `AZURE_OPENAI_DEPLOYMENT_NAME`, `AZURE_OPENAI_API_KEY`: For orchestrating agent
3. **Required packages** installed: `semantic-kernel`, `openai`

## Architecture Overview

```
┌─────────────────┐    ┌──────────────────┐    ┌─────────────────┐
│  Semantic       │    │   NIM Plugin     │    │  Local NIM      │
│  Kernel Agent   │◄──►│   (Function)     │◄──►│  Deployment     │
│                 │    │                  │    │                 │
└─────────────────┘    └──────────────────┘    └─────────────────┘
        │                                              │
        ▼                                              ▼
┌─────────────────┐                          ┌─────────────────┐
│ Azure OpenAI    │                          │ LLaMA 3.1 8B    │
│ (Orchestrator)  │                          │ Instruct Model  │
└─────────────────┘                          └─────────────────┘
```

# Configuration Section: Set Your Keys and Variables

**Important**: Update the values below with your actual configuration before running the rest of the notebook.

In [ ]:
# =====================================================
# CONFIGURATION: UPDATE THESE VALUES FOR YOUR SETUP
# =====================================================

# Local NIM Configuration
nim_endpoint_url = "http://localhost:8000"  # Your local NIM endpoint
nim_api_key = "your-nim-api-key"            # Your NIM API key (or set to None if not required)
nim_model_name = "meta/llama-3.1-8b-instruct"  # Model name in your NIM deployment

# Azure OpenAI Configuration (for orchestration agent)
azure_openai_endpoint = "https://your-resource.openai.azure.com"  # Your Azure OpenAI endpoint
azure_openai_deployment_name = "gpt-4"      # Your deployment name
azure_openai_api_key = "your-azure-openai-api-key"  # Your Azure OpenAI API key

# Alternative: Use OpenAI instead of Azure OpenAI
use_azure_openai = True  # Set to False to use regular OpenAI
openai_api_key = "your-openai-api-key"      # Only needed if use_azure_openai = False

# Advanced Configuration
nim_max_retries = 3        # Number of retry attempts for failed requests
nim_timeout_seconds = 30   # Timeout for NIM requests
nim_max_tokens = 256      # Maximum tokens for NIM responses
nim_temperature = 0.7     # Temperature for response generation

print("✅ Configuration loaded!")
print(f"🏠 NIM Endpoint: {nim_endpoint_url}")
print(f"🤖 NIM Model: {nim_model_name}")
print(f"🔵 Using Azure OpenAI: {use_azure_openai}")

# Validate configuration
config_warnings = []
if nim_endpoint_url == "http://localhost:8000":
    config_warnings.append("⚠️  Using default NIM endpoint - update if different")
if nim_api_key == "your-nim-api-key":
    config_warnings.append("⚠️  Update nim_api_key with your actual API key")
if use_azure_openai and azure_openai_endpoint.startswith("https://your-resource"):
    config_warnings.append("⚠️  Update Azure OpenAI configuration")
if not use_azure_openai and openai_api_key == "your-openai-api-key":
    config_warnings.append("⚠️  Update OpenAI API key")

if config_warnings:
    print("\n📋 Configuration Warnings:")
    for warning in config_warnings:
        print(f"  {warning}")
else:
    print("✅ All configuration looks good!")

# Section 1: Import All necessairy modules

In [ ]:
# Import required libraries for Semantic Kernel and NIM integration
import asyncio
import logging
import os
from typing import Annotated

# Semantic Kernel imports
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatCompletion
from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.open_ai_prompt_execution_settings import (
    OpenAIChatPromptExecutionSettings,
)
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.contents.function_call_content import FunctionCallContent
from semantic_kernel.functions.kernel_arguments import KernelArguments
from semantic_kernel.functions.kernel_function_decorator import kernel_function
from semantic_kernel.kernel import Kernel

# OpenAI client for NIM communication
from openai import OpenAI
from openai.types.chat import ChatCompletionUserMessageParam

# Configure logging for better debugging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

print("✅ All libraries imported successfully!")
print("📦 Semantic Kernel version:", "1.0.0+")  # Version check would be dynamic in real use
print("🔗 OpenAI client ready for NIM integration")

# Section 2: Connect to Local NIM Deployment

Configure the connection to your local NIM deployment. This includes setting up the endpoint URL, API key, and connection parameters.

In [ ]:
# Configuration for local NIM deployment
# Set these environment variables or update the values directly

# NIM endpoint configuration
nim_endpoint_url = "http://localhost:8000"
nim_api_key = "your-nim-api-key"

# Model configuration for NIM
nim_model_name = "meta/llama-3.1-8b-instruct"  # Default LLaMA 3.1 8B model

# NIM Connection Setup using configuration variables
# Uses the variables set in the Configuration Section above

nim_url = nim_endpoint_url + "/v1"

# Test connection to NIM
def test_nim_connection():
    """Test basic connectivity to the local NIM deployment."""
    try:
        if nim_endpoint_url.startswith("http://localhost") or nim_endpoint_url.startswith("http://127.0.0.1"):
            print(f"🔗 Connecting to local NIM deployment at: {nim_url}")
        else:
            print(f"🌐 Connecting to remote NIM deployment at: {nim_url}")
        
        # Validate configuration
        if nim_api_key == "your-nim-api-key":
            print("⚠️  Warning: Using default API key. Update nim_api_key in the Configuration Section.")
        
        # Create OpenAI client for NIM
        client = OpenAI(
            base_url=nim_url,
            api_key=nim_api_key
        )
        
        print(f"✅ NIM client configured successfully!")
        print(f"📊 Model: {nim_model_name}")
        return client
        
    except Exception as e:
        print(f"❌ Failed to configure NIM client: {str(e)}")
        return None

# Test the connection
nim_client = test_nim_connection()

# Section 3: Run a Simple Inference Example

Test your local NIM deployment with a direct inference request to verify it's working correctly.

In [ ]:
# Simple inference example with local NIM
def run_simple_inference(question: str) -> str:
    """
    Run a simple inference request against the local NIM deployment.
    
    Args:
        question (str): The question or prompt to send to the model
        
    Returns:
        str: The model's response
    """
    try:
        if nim_client is None:
            return "❌ NIM client not initialized. Please run the connection test first."
        
        print(f"🤔 Question: {question}")
        print(f"⏳ Sending request to local NIM...")
        
        # Create the message in OpenAI format
        messages = [
            ChatCompletionUserMessageParam(role="user", content=question)
        ]
        
        # Send request to NIM
        response = nim_client.chat.completions.create(
            model=nim_model_name,
            messages=messages,
            max_tokens=128,
            temperature=0.7,
            stream=False
        )
        
        # Extract and return the response
        result = response.choices[0].message.content or "No response from model"
        print(f"🎯 Response: {result}")
        return result
        
    except Exception as e:
        error_msg = f"❌ Error during inference: {str(e)}"
        print(error_msg)
        return error_msg

# Test with a simple question
test_question = "What are the key benefits of using NVIDIA GPUs for AI inference?"
response = run_simple_inference(test_question)

# Section 4: Create Semantic Kernel Plugin for NIM

Create a Semantic Kernel plugin that wraps your local NIM deployment, enabling it to be used as a function in AI orchestration workflows.

In [ ]:
class LocalNIMPlugin:
    """
    A Semantic Kernel plugin that provides access to local NIM deployment.
    Uses configuration variables set in the Configuration Section.
    
    This plugin wraps your local NIM instance and makes it available as a 
    Semantic Kernel function for use in AI workflows and agent conversations.
    """
    
    def __init__(self):
        """
        Initialize the NIM plugin using configuration variables.
        """
        self.nim_url = nim_url
        self.api_key = nim_api_key
        self.model_name = nim_model_name
        self.max_tokens = nim_max_tokens
        self.temperature = nim_temperature
        self.client = OpenAI(base_url=nim_url, api_key=nim_api_key)
        
    @kernel_function(
        name="get_nim_response",
        description="Get a response from the local NIM deployment for any question or task"
    )
    def get_nim_response(
        self, 
        question: Annotated[str, "The question or prompt to send to the NIM model"]
    ) -> Annotated[str, "The response from the NIM model"]:
        """
        Get a response from the local NIM deployment.
        
        Args:
            question (str): The question or prompt to process
            
        Returns:
            str: The model's response
        """
        try:
            logger.info(f"Processing NIM request: {question[:50]}...")
            
            # Clean the prompt (remove references to the plugin name)
            prompt = question.replace("nim", "you").replace("NIM", "you")
            
            # Validate inputs
            if not prompt.strip():
                return "❗ Please provide a valid question or prompt."
            
            # Check configuration
            if self.api_key == "your-nim-api-key":
                return "❗ Please update nim_api_key in the Configuration Section"
            
            logger.info(f"Sending request to NIM: {self.nim_url}")
            
            # Create message in OpenAI format
            messages = [
                ChatCompletionUserMessageParam(role="user", content=prompt)
            ]
            
            # Send request to local NIM
            response = self.client.chat.completions.create(
                model=self.model_name,
                messages=messages,
                max_tokens=self.max_tokens,
                temperature=self.temperature,
                stream=False
            )
            
            # Extract and return response
            result = response.choices[0].message.content or "No response from model"
            logger.info(f"Received response from NIM: {result[:50]}...")
            return result
            
        except Exception as e:
            error_msg = f"❗ Error calling local NIM: {str(e)}"
            logger.error(error_msg)
            return error_msg

# Create the NIM plugin instance using configuration variables
print("🔧 Creating Local NIM Plugin...")
local_nim_plugin = LocalNIMPlugin()
print("✅ Local NIM Plugin created successfully!")